In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

W0725 07:53:46.722522 140261770196864 deprecation.py:323] From <ipython-input-1-1d8ffff3e42a>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 07:53:46.724536 140261770196864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 07:53:46.728708 140261770196864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0725 07:53:46.964308 140261770196864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0725 07:53:47.007971 140261770196864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np

np.unique(mnist.train._labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

class RNN:
  def __init__(self, learning_rate):
    self.X = tf.placeholder(tf.float32, [None, 28, 28])
    self.Y = tf.placeholder(tf.int32, [None])
    
    def cell():
      return tf.nn.rnn_cell.BasicRNNCell(256)
    
    rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cell() for _ in range(2)])
    outputs, _ = tf.nn.dynamic_rnn(rnn_cells, self.X, dtype = tf.float32)
    
    self.logits = tf.layers.dense(outputs[:,-1], 10)
    self.cost = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels = self.Y, logits = self.logits)
    self.cost = tf.reduce_mean(self.cost)
    self.optimizer = tf.train.AdamOptimizer(learning_rate) \
    .minimize(self.cost)
    correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
    self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
learning_rate = 1e-3
batch_size = 128
epoch = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = RNN(learning_rate)
sess.run(tf.global_variables_initializer())

from tqdm import tqdm

for e in range(epoch):
  pbar = tqdm(
        range(0, len(mnist.train._labels), batch_size), desc = 'train loop'
    )
  train_loss, train_acc, test_loss, test_acc = [], [], [], []
  for i in pbar:
    index = min(i + batch_size, len(mnist.train._labels))
    batch_x = mnist.train._images[i: index]
    batch_x = np.reshape(batch_x, (-1, 28, 28))
    batch_y = mnist.train._labels[i: index]
    _, cost, accuracy = sess.run([model.optimizer, model.cost, model.accuracy],
                      feed_dict = {model.X: batch_x,
                                  model.Y: batch_y})
    train_loss.append(cost)
    train_acc.append(accuracy)
    pbar.set_postfix(cost = cost, accuracy = accuracy)

  pbar = tqdm(
        range(0, len(mnist.test._labels), batch_size), desc = 'test loop'
    )  
  for i in pbar:
    index = min(i + batch_size, len(mnist.test._labels))
    batch_x = mnist.test._images[i: index]
    batch_x = np.reshape(batch_x, (-1, 28, 28))
    batch_y = mnist.test._labels[i: index]
    cost, accuracy = sess.run([model.cost, model.accuracy],
                      feed_dict = {model.X: batch_x,
                                  model.Y: batch_y})
    test_loss.append(cost)
    test_acc.append(accuracy)
    pbar.set_postfix(cost = cost, accuracy = accuracy)
  
  
  print('\nepoch: %d, avg train loss: %f, avg train accuracy: %f'%(e + 1,
                                                             np.mean(train_loss),
                                                             np.mean(train_acc)))
  print('epoch: %d, avg test loss: %f, avg test accuracy: %f'%(e + 1,
                                                             np.mean(test_loss),
                                                             np.mean(test_acc)))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W0725 08:04:24.127211 140261770196864 deprecation.py:323] From <ipython-input-3-2ea81dfad9d8>:11: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0725 08:04:24.131228 140261770196864 deprecation.py:323] From <ipython-input-3-2ea81dfad9d8>:13: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0725 08:04:24.133150 140261770196864 deprecation.py:323] From <ipython-input-3-2ea81dfad9d8>:14: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API



epoch: 1, avg train loss: 0.401285, avg train accuracy: 0.875322
epoch: 1, avg test loss: 0.284025, avg test accuracy: 0.916634


train loop:   1%|          | 5/430 [00:00<00:09, 45.05it/s, accuracy=0.93, cost=0.185] 


epoch: 2, avg train loss: 0.198081, avg train accuracy: 0.940508
epoch: 2, avg test loss: 0.231354, avg test accuracy: 0.930676


...


epoch: 8, avg train loss: 0.097464, avg train accuracy: 0.971186
epoch: 8, avg test loss: 0.114028, avg test accuracy: 0.965981


train loop:   1%|▏         | 6/430 [00:00<00:08, 51.26it/s, accuracy=0.977, cost=0.0683]


epoch: 9, avg train loss: 0.110708, avg train accuracy: 0.967625
epoch: 9, avg test loss: 0.102293, avg test accuracy: 0.970728


test loop: 100%|██████████| 79/79 [00:00<00:00, 84.96it/s, accuracy=1, cost=0.00925]


epoch: 10, avg train loss: 0.094465, avg train accuracy: 0.972413
epoch: 10, avg test loss: 0.092329, avg test accuracy: 0.974189
